In [139]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [140]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [141]:
df=pd.read_csv('IMDB_data.csv')

In [142]:
df.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [143]:
df.shape

(250, 38)

In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         250 non-null    int64  
 1   Title              250 non-null    object 
 2   Year               250 non-null    int64  
 3   Rated              250 non-null    object 
 4   Released           248 non-null    object 
 5   Runtime            250 non-null    object 
 6   Genre              250 non-null    object 
 7   Director           250 non-null    object 
 8   Writer             249 non-null    object 
 9   Actors             250 non-null    object 
 10  Plot               250 non-null    object 
 11  Language           250 non-null    object 
 12  Country            250 non-null    object 
 13  Awards             245 non-null    object 
 14  Poster             250 non-null    object 
 15  Ratings.Source     250 non-null    object 
 16  Ratings.Value      250 non

In [145]:
df.columns

Index(['Unnamed: 0', 'Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre',
       'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards',
       'Poster', 'Ratings.Source', 'Ratings.Value', 'Metascore', 'imdbRating',
       'imdbVotes', 'imdbID', 'Type', 'tomatoMeter', 'tomatoImage',
       'tomatoRating', 'tomatoReviews', 'tomatoFresh', 'tomatoRotten',
       'tomatoConsensus', 'tomatoUserMeter', 'tomatoUserRating',
       'tomatoUserReviews', 'tomatoURL', 'DVD', 'BoxOffice', 'Production',
       'Website', 'Response'],
      dtype='object')

In [146]:
df=df[['Title','Genre','Director','Actors','Plot']]

In [147]:
df.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [148]:
df['Actors']=df['Actors'].map(lambda x:x.split(',')[:3])

In [149]:
df['Actors']

0            [Tim Robbins,  Morgan Freeman,  Bob Gunton]
1               [Marlon Brando,  Al Pacino,  James Caan]
2             [Al Pacino,  Robert Duvall,  Diane Keaton]
3        [Christian Bale,  Heath Ledger,  Aaron Eckhart]
4           [Martin Balsam,  John Fiedler,  Lee J. Cobb]
                             ...                        
245           [Ray Milland,  Jane Wyman,  Phillip Terry]
246    [Brie Larson,  John Gallagher Jr.,  Stephanie ...
247      [Cary Grant,  Rosalind Russell,  Ralph Bellamy]
248    [Sissy Spacek,  Jane Galloway Heitz,  Joseph A...
249           [Dev Patel,  Saurabh Shukla,  Anil Kapoor]
Name: Actors, Length: 250, dtype: object

In [150]:
df['Genre']=df['Genre'].map(lambda x:x.lower().split(','))

In [151]:
df['Genre']

0                 [crime,  drama]
1                 [crime,  drama]
2                 [crime,  drama]
3        [action,  crime,  drama]
4                 [crime,  drama]
                  ...            
245           [drama,  film-noir]
246                       [drama]
247    [comedy,  drama,  romance]
248           [biography,  drama]
249                       [drama]
Name: Genre, Length: 250, dtype: object

In [152]:
df['Director']=df['Director'].map(lambda x:x.split(' '))

In [153]:
df['Director']

0                      [Frank, Darabont]
1               [Francis, Ford, Coppola]
2               [Francis, Ford, Coppola]
3                   [Christopher, Nolan]
4                        [Sidney, Lumet]
                     ...                
245                      [Billy, Wilder]
246            [Destin, Daniel, Cretton]
247                      [Howard, Hawks]
248                       [David, Lynch]
249    [Danny, Boyle,, Loveleen, Tandan]
Name: Director, Length: 250, dtype: object

In [154]:
for index,row in df.iterrows():
    row['Actors']=[x.lower().replace(" ","")for x in row['Actors']]
    row['Director']=''.join(row['Director']).lower()

In [155]:
df['Actors']

0            [Tim Robbins,  Morgan Freeman,  Bob Gunton]
1               [Marlon Brando,  Al Pacino,  James Caan]
2             [Al Pacino,  Robert Duvall,  Diane Keaton]
3        [Christian Bale,  Heath Ledger,  Aaron Eckhart]
4           [Martin Balsam,  John Fiedler,  Lee J. Cobb]
                             ...                        
245           [Ray Milland,  Jane Wyman,  Phillip Terry]
246    [Brie Larson,  John Gallagher Jr.,  Stephanie ...
247      [Cary Grant,  Rosalind Russell,  Ralph Bellamy]
248    [Sissy Spacek,  Jane Galloway Heitz,  Joseph A...
249           [Dev Patel,  Saurabh Shukla,  Anil Kapoor]
Name: Actors, Length: 250, dtype: object

In [156]:
df['Director']

0                      [Frank, Darabont]
1               [Francis, Ford, Coppola]
2               [Francis, Ford, Coppola]
3                   [Christopher, Nolan]
4                        [Sidney, Lumet]
                     ...                
245                      [Billy, Wilder]
246            [Destin, Daniel, Cretton]
247                      [Howard, Hawks]
248                       [David, Lynch]
249    [Danny, Boyle,, Loveleen, Tandan]
Name: Director, Length: 250, dtype: object

In [157]:
!pip install rake_nltk

In [158]:
!pip install nltk

In [159]:
from rake_nltk import Rake

In [160]:
import nltk

In [161]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ELCOT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [162]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ELCOT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [163]:
df['key_words']=""
for index,row in df.iterrows():
    plot=row['Plot']
    r=Rake()
    r.extract_keywords_from_text(plot)
    key_words_dict_scores=r.get_word_degrees()
    row['key_words']=list(key_words_dict_scores.keys())
df.drop('Plot',axis=1,inplace=True)

In [164]:
key_words_dict_scores

defaultdict(<function rake_nltk.rake.Rake._build_word_co_occurance_graph.<locals>.<lambda>()>,
            {'mumbai': 3,
             'teen': 3,
             'reflects': 3,
             'upbringing': 1,
             'slums': 1,
             'accused': 1,
             'cheating': 1,
             'indian': 2,
             'version': 2,
             'wants': 1,
             'millionaire': 2,
             '?"': 2})

In [165]:
df.set_index('Title',inplace=True)

In [166]:
df.head()

,Genre,Director,Actors,key_words
Title,,,,
The Shawshank Redemption,"[crime, drama]","[Frank, Darabont]","[Tim Robbins, Morgan Freeman, Bob Gunton]",
The Godfather,"[crime, drama]","[Francis, Ford, Coppola]","[Marlon Brando, Al Pacino, James Caan]",
The Godfather: Part II,"[crime, drama]","[Francis, Ford, Coppola]","[Al Pacino, Robert Duvall, Diane Keaton]",
The Dark Knight,"[action, crime, drama]","[Christopher, Nolan]","[Christian Bale, Heath Ledger, Aaron Eckhart]",
12 Angry Men,"[crime, drama]","[Sidney, Lumet]","[Martin Balsam, John Fiedler, Lee J. Cobb]",


In [168]:
df['bag_of_words'] = ""  

columns = df.columns

for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'Director':
            if isinstance(row[col], list):  
                words += ' '.join(row[col]) + ' '
            else:
                words += str(row[col]) + ' '  
        else:
            words += str(row[col]) + ' '  
    df.at[index, 'bag_of_words'] = words.strip()  

df = df[['bag_of_words']]  # Keep only 'bag_of_words' column

In [170]:
df.head()

,bag_of_words
Title,
The Shawshank Redemption,"crime drama ['Frank', 'Darabont'] Tim Robbins..."
The Godfather,"crime drama ['Francis', 'Ford', 'Coppola'] Ma..."
The Godfather: Part II,"crime drama ['Francis', 'Ford', 'Coppola'] Al..."
The Dark Knight,"action crime drama ['Christopher', 'Nolan'] ..."
12 Angry Men,"crime drama ['Sidney', 'Lumet'] Martin Balsam..."


In [172]:
count=CountVectorizer()
count_matrix=count.fit_transform(df['bag_of_words'])
indices=pd.Series(df.index)
indices[:5]

0    The Shawshank Redemption
1               The Godfather
2      The Godfather: Part II
3             The Dark Knight
4                12 Angry Men
Name: Title, dtype: object

In [174]:
c=count_matrix.todense()

In [176]:
cosine_sim=cosine_similarity(count_matrix,count_matrix)

In [186]:
def recommendations(title,codine_sim=cosine_sim):
    recommended_movies=[]
    idx=indices[indices==title].index[0]
    score_series=pd.Series(cosine_sim[idx]).sort_values(ascending=False)
    top_10_indexes=list(score_series.iloc[1:11].index)
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
    return recommended_movies

In [188]:
recommendations('Fargo')

['No Country for Old Men',
 'The Big Lebowski',
 'Network',
 'Rope',
 'The Silence of the Lambs',
 'Reservoir Dogs',
 'On the Waterfront',
 'Papillon',
 'Léon: The Professional',
 'The Departed']